In [6]:
# Open the file in write mode
classes_file = open("habitforge_classes.py", "w")

# Define the class contents as a string
class_content = """import re
import random
from datetime import date, timedelta
import matplotlib.pyplot as plt

import re
import random
from datetime import date, timedelta
import matplotlib.pyplot as plt

# Defining neccessary classes
# USER class

class User:
    
    def __init__(self, user_ID, first, last, email):
        self.user_ID = user_ID
        self.name = first + ' ' + last
        self.email = email
        self.habits = []  # List to store user habits
        self.log_entries = []  # List to store users log entries
    
    def __str__(self):
        return(f"User(ID: {self.user_ID}, Name: {self.name}, Email: {self.email})")
    
    # Function to validate users email address
    
    def validate_email(email: str) -> bool:
        email_regex = r"[^@]+@[^@]+\.[^@]+"
        return re.match(email_regex, email) is not None
    
    # Function to generate a random user id
    
    def generate_user_id():
        return random.randint(1000, 9999)

    def calculate_analytics(self):
        habit_names = []
        current_streaks = []
        max_streaks = []
        completion_rates = []

        for habit in self.habits:
            habit_names.append(habit.name)
            current_streak = habit.calculate_streak(self.log_entries)
            current_streaks.append(current_streak)
            max_streak = habit.calculate_max_streak(self.log_entries)
            max_streaks.append(max_streak)
            completion_rate = habit.calculate_completion_rate(self.log_entries)
            completion_rates.append(completion_rate)

        self.plot_analytics(habit_names, current_streaks, max_streaks, completion_rates)
    
    def plot_analytics(self, habit_names, current_streaks, max_streaks, completion_rates):
        fig, axs = plt.subplots(3, 1, figsize=(10, 15))

        # Current Streaks
        axs[0].bar(habit_names, current_streaks, color='blue')
        axs[0].set_title('Current Streaks')
        axs[0].set_ylabel('Streak Length')

        # Maximum Streaks
        axs[1].bar(habit_names, max_streaks, color='green')
        axs[1].set_title('Maximum Streaks')
        axs[1].set_ylabel('Streak Length')

        # Completion Rates
        axs[2].bar(habit_names, completion_rates, color='red')
        axs[2].set_title('Completion Rates')
        axs[2].set_ylabel('Completion Rate (%)')

        for ax in axs:
            ax.set_xlabel('Habits')
            ax.set_xticklabels(habit_names, rotation=45, ha='right')

        plt.tight_layout()
        plt.show()

# HABIT class

class Habit:
    
    def __init__(self, user_ID, habit_ID, name, frequency):
        self.user_ID = user_ID
        self.habit_ID = habit_ID
        self.name = name
        self.frequency = frequency
        
    def __str__(self):
        return(f"Habit(ID: {self.habit_ID}, Name: {self.name}, Frequency: {self.frequency}, User ID: {self.user_ID})")

   # Function to edit habits and update the habits list

    def update_habit(self):
        name = input("Enter new habit name (or press Enter to keep current): ")
        frequency = input("Enter new habit frequency (Daily, Weekly, or Monthly) (or press Enter to keep current): ").title()
        if frequency and frequency not in ['Daily', 'Weekly', 'Monthly']:
            print("Invalid frequency. Must be Daily, Weekly, or Monthly")
        else:
            if name:
                self.name = name
            if frequency:
                self.frequency = frequency
        print(f"Habit updated: {self}")
    
    # Function to delete habit
    
    def delete(user, habit_ID):
        user.habits = [habit for habit in user.habits if habit.habit_ID != habit_ID]
    
    # Function to calculate streaks
    
    def calculate_streak(self, log_entries):
        sorted_entries = sorted([log for log in log_entries if log.habit_ID == self.habit_ID], key=lambda x: x.date)
        streak = 0
        if not sorted_entries:
            return streak

        if self.frequency == "Daily":
            streak = self._calculate_daily_streak(sorted_entries)
        elif self.frequency == "Weekly":
            streak = self._calculate_weekly_streak(sorted_entries)
        elif self.frequency == "Monthly":
            streak = self._calculate_monthly_streak(sorted_entries)
        
        return streak

    # Function to calculate daily streaks
    
    def _calculate_daily_streak(self, sorted_entries):
        streak = 0
        current_streak = 0
        previous_date = sorted_entries[0].date - timedelta(days=1)
        for entry in sorted_entries:
            if entry.status == "Done" and entry.date == previous_date + timedelta(days=1):
                current_streak += 1
                streak = max(streak, current_streak)
            elif entry.status == "Done":
                current_streak = 1
            else:
                current_streak = 0
            previous_date = entry.date
        return streak

    # Function to calculate weekly streaks
    
    def _calculate_weekly_streak(self, sorted_entries):
        streak = 0
        current_streak = 0
        previous_date = sorted_entries[0].date - timedelta(weeks=1)
        for entry in sorted_entries:
            if entry.status == "Done" and entry.date == previous_date + timedelta(weeks=1):
                current_streak += 1
                streak = max(streak, current_streak)
            elif entry.status == "Done":
                current_streak = 1
            else:
                current_streak = 0
            previous_date = entry.date
        return streak

    # Function to calculate monthly streaks
    
    def _calculate_monthly_streak(self, sorted_entries):
        streak = 0
        current_streak = 0
        previous_date = sorted_entries[0].date.replace(day=1) - timedelta(days=1)
        for entry in sorted_entries:
            next_month = previous_date.replace(day=28) + timedelta(days=4)
            if entry.status == "Done" and entry.date.replace(day=1) == next_month.replace(day=1):
                current_streak += 1
                streak = max(streak, current_streak)
            elif entry.status == "Done":
                current_streak = 1
            else:
                current_streak = 0
            previous_date = entry.date
        return streak

    def calculate_max_streak(self, log_entries):
        sorted_entries = sorted([log for log in log_entries if log.habit_ID == self.habit_ID], key=lambda x: x.date)
        streak = 0
        max_streak = 0
        if not sorted_entries:
            return max_streak

        if self.frequency == "Daily":
            streak, max_streak = self._calculate_max_daily_streak(sorted_entries)
        elif self.frequency == "Weekly":
            streak, max_streak = self._calculate_max_weekly_streak(sorted_entries)
        elif self.frequency == "Monthly":
            streak, max_streak = self._calculate_max_monthly_streak(sorted_entries)
        
        return max_streak

    # Function to calculate maximum DAILY streak
    
    def _calculate_max_daily_streak(self, sorted_entries):
        streak = 0
        max_streak = 0
        previous_date = sorted_entries[0].date - timedelta(days=1)
        for entry in sorted_entries:
            if entry.status == "Done" and entry.date == previous_date + timedelta(days=1):
                streak += 1
                max_streak = max(max_streak, streak)
            elif entry.status == "Done":
                streak = 1
                max_streak = max(max_streak, streak)
            else:
                streak = 0
            previous_date = entry.date
        return streak, max_streak

    # Function to calculate maximum WEEKLY streak
    
    def _calculate_max_weekly_streak(self, sorted_entries):
        streak = 0
        max_streak = 0
        previous_date = sorted_entries[0].date - timedelta(weeks=1)
        for entry in sorted_entries:
            if entry.status == "Done" and entry.date == previous_date + timedelta(weeks=1):
                streak += 1
                max_streak = max(max_streak, streak)
            elif entry.status == "Done":
                streak = 1
                max_streak = max(max_streak, streak)
            else:
                streak = 0
            previous_date = entry.date
        return streak, max_streak

    # Function to calculate maximum MONTHLY streak
    
    def _calculate_max_monthly_streak(self, sorted_entries):
        streak = 0
        max_streak = 0
        previous_date = sorted_entries[0].date.replace(day=1) - timedelta(days=1)
        for entry in sorted_entries:
            next_month = previous_date.replace(day=28) + timedelta(days=4)
            if entry.status == "Done" and entry.date.replace(day=1) == next_month.replace(day=1):
                streak += 1
                max_streak = max(max_streak, streak)
            elif entry.status == "Done":
                streak = 1
                max_streak = max(max_streak, streak)
            else:
                streak = 0
            previous_date = entry.date
        return streak, max_streak

    # Function to calculate completion rate
    
    def calculate_completion_rate(self, log_entries):
        total_entries = sum(1 for log in log_entries if log.habit_ID == self.habit_ID)
        done_entries = sum(1 for log in log_entries if log.habit_ID == self.habit_ID and log.status == "Done")
        if total_entries == 0:
            return 0
        return (done_entries / total_entries) * 100

# LOG-ENTRY class

class LogEntry:
    
    def __init__(self, log_ID, date, status, habit_ID):
        self.log_ID = log_ID
        self.date = date
        self.status = status
        self.habit_ID = habit_ID
        
    def __str__(self):
        return(f"Log(ID: {self.log_ID}, Date: {self.date}, Status: {self.status}, Habit ID: {self.habit_ID})")
    
    # Function to add a new log entry
    
    def add_log_entry():
        h_id = input("Enter Habit ID: ")
        while True:
            if not any(habit.habit_ID == h_id for habit in User.habits):
                print("ID Does not exist. Please try again.")
                h_id = int(input("Enter Habit ID: "))
            else:
                break
        status = input("Enter log status (Done or Missed)").title()
        while True:
            if status in ['Done', 'Missed']:
                break
            else:
                print("Invalid status. Must be either \'Done\' or \'Missed\'")
        log_ID = len(new_user.log_entries)+1 
        new_log_entry = LogEntry(log_ID, date.today(), status, h_id)
        return new_log_entry
"""

# Write the class contents to the file
classes_file.write(class_content)

# Close the file
classes_file.close()